In [78]:
%matplotlib inline

import numpy as np
import os,sys
from PIL import Image
from helpers import *
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from pylab import *

# Loading of the training images 

In [95]:
# Loaded a set of images

root_dir = "augmented_training/"
#root_dir = "training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)  

n = 800

print("Loading " + str(n) + " satellite images")
imgs = [load_image(image_dir + files[i]) for i in range(n)] #List of each image Dim (100, 400,400,3)

gt_dir = root_dir + "groundtruth/"
files2 = os.listdir(gt_dir)

print("Loading " + str(n) + " groundtruth images")
gt_imgs = [load_image(gt_dir + files2[i]) for i in range(n)]
print('Image size = ' + str(imgs[0].shape[0]) + ',' + str(imgs[0].shape[1]))

Loading 800 satellite images
Loading 800 groundtruth images
Image size = 400,400


# Extract patches from input images

In [96]:
patch_size = 16 # each patch is 16*16 pixels  #400/16 = 25*25 = 625 patches for 10 images

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]   # dim img_patches = (n,625,16,16,3)
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches

img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

print("img_patches:" + str(img_patches.shape))
print("gt_patches:" + str(gt_patches.shape))

img_patches:(500000, 16, 16, 3)
gt_patches:(500000, 16, 16)


# Building of the Dataset & Polynomial

In [97]:
def build_X_Y_classes(img_patches,gt_patches,foreground_threshold):
    
    X = np.asarray([extract_features(img_patches[i]) for i in range(len(img_patches))])
    Y = np.asarray([value_to_class(np.mean(gt_patches[i]),foreground_threshold) for i in range(len(gt_patches))])
    
    return X,Y

def build_poly(X,n):
    
    poly = PolynomialFeatures(n, interaction_only=False) #Including interaction 
    return poly.fit_transform(X)


# Logistic regression 

In [99]:
from sklearn.metrics import f1_score

def logistic_regression_test(degree):
    
        foreground_threshold = 0.25
        t1 = datetime.datetime.now()
        X,Y = build_X_Y_classes(img_patches,gt_patches,foreground_threshold) 
        t2 = datetime.datetime.now()
        
        print("Build classes time: ",t2 - t1)
        
        tX = build_poly(X,degree)
        logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced",max_iter = 100)
    
        t3 = datetime.datetime.now()
        logreg.fit(tX,Y)
        t4 = datetime.datetime.now()
        
        print("Log_reg time: ", t4 - t3)
        
    # Predict on the training set
        Z = logreg.predict(tX)

        ratio = f1_score(Y,Z, average = 'macro')
        
        weight = logreg.coef_
        print('Ratio = ' + str(ratio))

        return ratio, weight 


## Test for different degrees of polynomials

In [100]:
ratios = []
weights = []
degrees = range(1,5)

for n in degrees:
    print("Degree {}\n".format(n))
          
    ratio, weight = logistic_regression_test(n)
          
    ratios.append(ratio)
    weights.append(weight)
    
print("\nMax ratio {a} at degree {b}".format(a = max(ratios) , b = ratios.index(max(ratios)) + 1)) 
    

Degree 1

Build classes time:  0:02:01.227545
Log_reg time:  0:00:02.349471
Ratio = 0.563814672431
Degree 2

Build classes time:  0:01:40.117842
Log_reg time:  0:00:36.721641
Ratio = 0.673430983951
Degree 3

Build classes time:  0:01:33.606131
Log_reg time:  0:03:19.439735
Ratio = 0.681667190235
Degree 4

Build classes time:  0:01:39.028671
Log_reg time:  0:09:56.862041
Ratio = 0.68325067749

Max ratio 0.6832506774902665 at degree 4


In [101]:
max_index = ratios.index(max(ratios))
weight = weights[max_index]

print(weight)

[[  1.04955372e+00  -9.07297527e+01  -3.02127799e+01   8.45424605e+01
   -5.41809764e+02   2.91029357e+02   2.61706893e+02  -1.78574225e+03
    3.55108864e+03   1.31629292e+03  -6.20545435e+02  -3.10291818e+02
   -2.24847120e+02  -8.59320248e+02  -1.59502228e+03  -3.03466442e+03
    2.15429481e+03   1.51091401e+03  -5.01294986e+02   7.10277671e+03
   -3.74962966e+03  -2.71105667e+03   1.42396738e+03  -1.63715837e+03
    2.17416004e+03  -2.90679285e+03  -7.21907063e+02   3.65997876e+02
   -1.03605633e+03  -4.58478218e+03   2.41682169e+03   1.27839663e+03
    1.62138701e+03  -3.67245674e+02   1.09500416e+02   3.51478180e+03
   -1.85861828e+03  -3.49670397e+02  -9.72902161e+02   2.71235591e+03
   -4.45912857e+02  -1.21701377e+03  -1.36683140e+03  -1.48911888e+03
   -7.30415294e+02  -2.18105489e+02   3.23529535e+02   1.63852951e+02
   -5.41040945e+02  -1.39541592e+00  -5.37332467e+02  -1.67955778e+03
    1.02814419e+03   1.19033038e+03   3.89933571e+02  -5.13589818e+02
   -3.82845517e+02  

## Loading of Test Images 

In [102]:
test_dir = 'test_set_images/'
imgs = load_test_images(test_dir) ##In helpers.py

Loading 50 test images
Size of image: 608,608


## Building of the Test dataset + polynomial

In [103]:
patch_size = 16 # each patch is 16*16 pixels  #400/16 = 25*25 = 625 patches for 10 images

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(len(imgs))]# dim img_patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])

print("img_patches:" + str(img_patches.shape))

data = np.asarray([extract_features(img_patches[i]) for i in range(len(img_patches))])
print('Dataset (RGB features Mean/Variance) :', data.shape)

t_data = build_poly(data,4)

img_patches:(72200, 16, 16, 3)
Dataset (RGB features Mean/Variance) : (72200, 6)


## Make the predictions

In [104]:
y_pred = t_data @ weight.T

for i in range(len(y_pred)):
    if y_pred[i] < 0.5:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

print('prediction_shape: before',y_pred.shape)
labels = np.reshape(y_pred,(50,1444))
print('prediction_shape: after',labels.shape) ##iterate over each image more easily


prediction_shape: before (72200, 1)
prediction_shape: after (50, 1444)


## Save the images 

In [105]:
imgwidth = imgs[0].shape[0] 
imgheight = imgs[0].shape[1] 
w = patch_size
h = patch_size

os.makedirs('prediction_test_logistic/', exist_ok=True) 

for i in range(len(labels)):
    im = label_to_img(imgwidth, imgheight, w, h, labels[i])
    im = img_float_to_uint8(im)
    Image.fromarray(im).save('prediction_test_logistic/prediction_' + '%.3d' % (i + 1)+ '.png')
    

## Mask to submission 

In [ ]:
'RUN mask_to_submission.py with the right path to test prediction'

# Other stuff

In [ ]:
patch_size = 16

def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
    return X

# Run prediction on the img_idx-th image
img_idx = 12

Xi = extract_img_features(image_dir + files[img_idx])
tXi = build_poly(Xi,3)
Zi = logreg.predict(tXi)
print(Zi.shape)
plt.scatter(Xi[:, 0], Xi[:, 4], c=Zi, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)


In [ ]:
# Plot features using predictions to color datapoints
plt.scatter(X[:, 0], X[:, 4], c=Z, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:

# Plot 2d features using groundtruth to color the datapoints

subplot(1,3,1)
subplots_adjust(left=None, bottom=None, right=2, top=None, wspace=None, hspace=None)
plt.scatter(X[:, 0], X[:, 3], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Red")
subplot(1,3,2)
plt.scatter(X[:, 1], X[:, 4], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Green")
subplot(1,3,3)
plt.scatter(X[:, 2], X[:, 5], c=Y, edgecolors='k', cmap=plt.cm.Paired) #c is the color  #orange dot = class 1 = foreground
plt.title("Blue")

In [ ]:
np.mean(gt_patches[4])

In [ ]:
extract_features(img_patches[4])

In [33]:
np.linspace(1,10,10)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

In [6]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import numpy as np

A = np.array([1,2,3])


array([1, 2, 3])